# Wine Quality

## I. Chuẩn bị dữ liệu

In [76]:
# Import các thư viện cần thiết
import os
import config as CONFIG
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#### 1. Lấy dữ liệu và xử lí thô

In [77]:
# Đọc dữ liệu rượu trắng
data_white = pd.read_csv(CONFIG.RAW_DATA_PATH_WHITE, delimiter=';')
#(R) data_white['wine_type'] = 'white'  # Thêm cột phân loại loại rượu

# Đọc dữ liệu rượu đỏ
#data_red = pd.read_csv(CONFIG.RAW_DATA_PATH_RED, delimiter=';')
#(R) data_red['wine_type'] = 'red'  # Thêm cột phân loại loại rượu

# Ghép hai dữ liệu
data = data_white # pd.concat([data_white, data_red], axis=0, ignore_index=True)

# Thêm cột "Quality_Category" để nhóm chất lượng
# 0-4: Chất lượng thấp, 5-6: Tiêu chuẩn, 7-10: Chất lượng cao
data['Quality_Category'] = pd.cut(
    data['quality'],
    bins=[-1, 4, 6, 10],
    labels=['Low', 'Standard', 'High']
)

- Kiểm tra dữ liệu

In [ ]:
# Kiểm tra thông tin dữ liệu
print(data.info())
# Kiểm tra sơ bộ
print(data.head())
# Kiểm tra giá trị duy nhất trong nhãn:
print(data['Quality_Category'].value_counts())
# Kiểm tra thống kê dữ liệu:
print(data.describe())

#### 2. Tiền xử lý dữ liệu

- Mã hóa 'wine_type' thành giá trị số 

In [79]:
# # -> có thể bỏ bước này, coi như dữ liệu không có phân loại rượu
#(R) encoder = LabelEncoder()
#(R) data['wine_type'] = encoder.fit_transform(data['wine_type'])

- Loại bỏ cột "quality" (không cần thiết)

In [80]:
data = data.drop(columns=['quality'])

- Xáo trộn dữ liệu

In [81]:
rand_value = CONFIG.RANDOM_STATE
data = shuffle(data, random_state=rand_value)

- Phân loại

In [82]:
X = data.drop(columns=['Quality_Category'])

y = data['Quality_Category']

# Chuyển đổi nhãn thành số
encoder = LabelEncoder()
y = encoder.fit_transform(y)

#### 3. Phân chia dữ liệu với các tỷ lệ khác nhau

In [83]:
splits = CONFIG.SPLIT_RATIOS
datasets = {}

In [84]:
for split_name, train_ratio in splits.items():
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_ratio, stratify=y, random_state=rand_value
    )
    datasets[split_name] = {
        'feature_train': X_train,
        'label_train': y_train,
        'feature_test': X_test,
        'label_test': y_test,
    }

#### 4. Lưu dữ liệu vào CSV

In [85]:
# Tạo thư mục cha chứa các file path
dir_path = CONFIG.PROCESSED_CSV_DIR
os.makedirs(dir_path, exist_ok=True)

# Tạo các thư mục chính chứa file path
for split_name in splits.keys():
    os.makedirs(
        os.path.join(dir_path, split_name.replace('/', '_')), exist_ok=True
        )

In [ ]:
try:
    # Lặp qua các phân chia trong datasets
    for split_name, dataset in datasets.items():
        # Đảm bảo rằng split_name là hợp lệ trong splits
        if split_name in splits:
            split_dir = os.path.join(dir_path, split_name.replace('/', '_'))
            
            # Lưu các dữ liệu vào các file CSV
            dataset['feature_train'].to_csv(os.path.join(split_dir, "train_features.csv"), index=False)
            pd.DataFrame(dataset['label_train']).to_csv(os.path.join(split_dir, "train_labels.csv"), index=False)
            dataset['feature_test'].to_csv(os.path.join(split_dir, "test_features.csv"), index=False)
            pd.DataFrame(dataset['label_test']).to_csv(os.path.join(split_dir, "test_labels.csv"), index=False)

    print("Lưu dữ liệu thành công!")

except Exception as e:
    print(f"Không thể lưu dữ liệu! Lỗi: {e}")

#### 5. Hình dung phân phối nhãn

In [87]:
order = encoder.classes_
def plot_label_distribution(y, title):
    y_decoded = encoder.inverse_transform(y)
    sns.countplot(x=y_decoded,order=order)
    plt.title(title)
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.show()

- Phân phối dữ liệu gốc

In [ ]:
plot_label_distribution(y, 'Original Dataset')

- Phân phối các tập con

In [ ]:
for split_name, dataset in datasets.items():
    plot_label_distribution(dataset['label_train'], f'Train Dataset ({split_name})')
    plot_label_distribution(dataset['label_test'], f'Test Dataset ({split_name})')

## 2.2 Xây dựng bộ phân loạii

In [90]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, accuracy_score
import graphviz

- Hàm huấn luyện và trực quan hóa cây quyết định

In [91]:
dir_path = CONFIG.RESULTS_BASE_TREE_DIR

def train_and_visualize(features_train, labels_train, features_test, labels_test, split_name):
    # Huấn luyện cây quyết định
    model = DecisionTreeClassifier(criterion='entropy', random_state=rand_value)
    model.fit(features_train, labels_train)
    
    # Dự đoán trên tập kiểm tra
    predictions = model.predict(features_test)
    
    # Đánh giá hiệu suất mô hình
    labels_test_decoded = encoder.inverse_transform(labels_test)
    predictions_decoded = encoder.inverse_transform(predictions)

    print(f"Results for split {split_name}")
    print("- Classification Report:")
    print(classification_report(labels_test_decoded, predictions_decoded))

    accuracy = accuracy_score(labels_test, predictions)
    print(f"Accuracy: {accuracy:.6f}\n")

    # Trực quan hóa cây quyết định
    dot_data = export_graphviz(
        model,
        out_file=None,
        feature_names=features_train.columns,
        class_names=encoder.classes_,
        filled=True,
        rounded=True,
        special_characters=True
    )
    graph = graphviz.Source(dot_data)
    filepath = f"{dir_path}/{split_name.replace('/', '_')}/graph"
    graph.render(filepath, format="pdf")
    # graph.view()  # Mở cây trực tiếp
    # display(graph)
    print(f"- Saved tree visualization for {split_name}")
    print("------------------------------------------\n")

- Huấn luyện và trực quan hóa với từng tập con

In [ ]:
for split_name, dataset in datasets.items():
    train_and_visualize(
        features_train=dataset['feature_train'],
        labels_train=dataset['label_train'],
        features_test=dataset['feature_test'],
        labels_test=dataset['label_test'],
        split_name=split_name
    )

## 2.3 Đánh giá các bộ phân loại

In [93]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

- Hàm đánh giá mô hình và tạo báo cáo

In [94]:
def evaluate_model(features_test, labels_test, model, split_name):
    # Dự đoán trên tập kiểm tra
    predictions = model.predict(features_test)
    
     # Chuyển ngược về nhãn
    labels_test_decoded = encoder.inverse_transform(labels_test)
    predictions_decoded = encoder.inverse_transform(predictions)

    # Classification Report
    print("------------------------------------------\n")
    print(f"- Classification Report for split {split_name}")
    print(classification_report(labels_test_decoded, predictions_decoded))
    # print(classification_report(labels_test, predictions))
    
    # Confusion Matrix
    cm = confusion_matrix(labels_test, predictions)
    print(f"- Confusion Matrix for split {split_name}:\n{cm}")
    
    # Trực quan hóa ma trận nhầm lẫn
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
    plt.title(f"Confusion Matrix for {split_name}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # Trả về mô hình và độ chính xác
    accuracy = accuracy_score(labels_test, predictions)
    return accuracy

- Lặp qua từng tập con để huấn luyện, đánh giá, và lưu kết quả

In [ ]:
results = {}
for split_name, dataset in datasets.items():
    # Huấn luyện mô hình
    model = DecisionTreeClassifier(criterion='entropy', random_state=rand_value)
    model.fit(dataset['feature_train'], dataset['label_train'])
    
    # Đánh giá mô hình
    accuracy = evaluate_model(
        features_test=dataset['feature_test'],
        labels_test=dataset['label_test'],
        model=model,
        split_name=split_name
    )
    
    # Lưu kết quả
    results[split_name] = accuracy

- Hiển thị tổng hợp kết quả độ chính xác

In [ ]:
for split_name, accuracy in results.items():
    print(f"Accuracy for {split_name}: {accuracy:.6f}")

## IV Độ sâu và độ chính xác

- Lấy bộ dữ liệu huấn luyện và kiểm tra 80/20

In [97]:
default_split = CONFIG.DEFAULT_SPLIT

X_train = datasets[default_split]['feature_train']
y_train = datasets[default_split]['label_train']
X_test  = datasets[default_split]['feature_test']
y_test  = datasets[default_split]['label_test']

- Thử nghiệm với các giá trị max_depth khác nhau

In [ ]:
accuracy_scores = {}
max_depth_values = CONFIG.MAX_DEPTH_VALUES
dir_path = CONFIG.RESULTS_TREE_MAX_DEPTH_DIR

for max_depth in max_depth_values:
    # Tạo và huấn luyện cây quyết định với max_depth
    model = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, random_state=rand_value)
    model.fit(X_train, y_train)
    
    # Dự đoán và tính độ chính xác
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores[max_depth] = accuracy
    
    # Trực quan hóa cây quyết định
    print(f"Decision Tree with max_depth={max_depth}")
    dot_data = export_graphviz(
        model,
        out_file=None,
        feature_names=X_train.columns,
        class_names=encoder.classes_,
        filled=True,
        rounded=True,
        special_characters=True
    )
    graph = graphviz.Source(dot_data)
    path = f"{dir_path}/max_depth_{'None' if max_depth is None else max_depth}/graph"
    graph.render(path, format="pdf")
    print(f"- Saved tree visualization for {path}.pdf")
    print("------------------------------------------\n")


- Hiển thị bảng kết quả

In [ ]:
print("max_depth\tAccuracy")
for max_depth, accuracy in accuracy_scores.items():
    print(f"{'None' if max_depth is None else max_depth}\t\t{accuracy:.6f}")

- Vẽ biểu đồ độ chính xác

In [ ]:
depth_labels = ['None' if d is None else d for d in max_depth_values]
accuracy_values = list(accuracy_scores.values())

plt.figure(figsize=(10, 6))
plt.plot(depth_labels, accuracy_values, marker='o', linestyle='-', color='b')
plt.title("Accuracy vs max_depth")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.grid(True)
for i, txt in enumerate(accuracy_values):
    plt.text(i, accuracy_values[i], f"{accuracy_values[i]:.6f}", fontsize=6, ha='center')

plt.show()